In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import json
import logging
import time
import random
from datetime import datetime
from prometheus_client import CollectorRegistry, Gauge, push_to_gateway
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# ✅ Log Configuration
LOG_FILE = "C:/Users/91966/ml_monitor/logs/ml_model.log"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def log_event(level, message):
    """Logs an event in JSON format."""
    log_entry = {"timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "level": level, "message": message}
    with open(LOG_FILE, "a") as f:
        f.write(json.dumps(log_entry) + "\n")
    logging.info(message) if level.lower() == "info" else logging.error(message)

# ✅ Load Iris Dataset
iris_data = load_iris()
df = pd.DataFrame(iris_data['data'], columns=iris_data['feature_names'])

# ✅ Encode Labels & Train-Test Split
x = iris_data['data']
y_ = iris_data['target'].reshape(-1, 1)  
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y_)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

# ✅ Define Model
model = Sequential([
    Dense(10, input_shape=(4,), activation='relu', name='fc1'),
    Dense(10, activation='relu', name='fc2'),
    Dense(3, activation='softmax', name='output')
])
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae'])
print(model.summary())

# ✅ Train Model with Logging Callback
class LoggingCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        log_event("info", f"Epoch {epoch+1}: Loss={logs['loss']:.4f}, Val_Loss={logs.get('val_loss', 0):.4f}, MAE={logs['mae']:.4f}")

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(train_x, train_y, epochs=100, verbose=1, validation_split=0.1, callbacks=[early_stop, LoggingCallback()])

# ✅ Log Training Results
hist = history.history
rmse_final = np.sqrt(float(hist['val_mse'][-1]))
log_event("info", f"Final RMSE on validation set: {rmse_final:.4f}")

# ✅ Evaluate Model
mse, mae, _ = model.evaluate(test_x, test_y)
rmse = np.sqrt(mse)
# log_event("info", f"Test Evaluation -> RMSE: {rmse:.4f}, MAE: {mae:.4f} , Z_Score = {z_scores} , Mean_Value = {mean_value}")

# ✅ Prometheus Metrics Setup
registry = CollectorRegistry()
n_rmse_gauge = Gauge("n_rmse", "Root Mean Squared Error", registry=registry)
accuracy_gauge = Gauge("accuracy", "Model Accuracy", registry=registry)
mean_metric = Gauge('feature_x_mean', 'Mean of feature_x', ['feature'], registry=registry)
stddev_metric = Gauge('feature_x_stddev', 'Standard Deviation of feature_x', ['feature'], registry=registry)
outlier_metric = Gauge('feature_x_outliers', 'Percentage of outliers in feature_x', ['feature'], registry=registry)
class_distribution = Gauge('class_distribution', 'Distribution of classes', ['species'], registry=registry)

# ✅ Compute Feature Statistics

# ✅ Send Metrics to Prometheus Pushgateway
try:
    push_to_gateway("localhost:9091", job="model_metrics", registry=registry)
    log_event("info", "📡 Feature drift metrics pushed to Prometheus!")
except Exception as e:
    log_event("error", f"⚠️ Error pushing metrics: {e}")

# ✅ Real-Time Monitoring & Logging
num = np.shape(test_y)[0]
push_interval = 10  
last_push = time.time()

for i in range(100000):  
    indx = random.randint(0, num - 1)  
    x_batch = np.expand_dims(test_x[indx, :], axis=0)
    y_batch = np.expand_dims(test_y[indx], axis=0)
    mse, mae, _ = model.evaluate(x_batch, y_batch, verbose=0)
    rmse = np.sqrt(mse)

    # ✅ Log real-time prediction performance

    # ✅ Push Metrics to Prometheus
    n_rmse_gauge.set(rmse)
    accuracy_gauge.set(1 / (1 + rmse)) 

    feature_stats = {}
    for feature in df.columns:
        mean_value = df[feature].mean()
        stddev_value = df[feature].std()
        z_scores = (df[feature] - mean_value) / stddev_value
        outlier_percentage = (np.abs(z_scores) > 3).sum() / len(df) * 100  
        feature_stats[feature] = {"mean": mean_value, "stddev": stddev_value, "outliers": outlier_percentage}

    # ✅ Compute Class Distribution
    species_labels = iris_data["target"]
    class_counts = pd.Series(species_labels).value_counts(normalize=True) * 100  # % Distribution
    
    log_event("info", f"Iteration {i}: RMSE={rmse:.4f}, MAE={mae:.4f} , Z_Score = {z_scores} , Mean_Value = {mean_value}")
    # ✅ Push Feature Metrics
    for feature, stats in feature_stats.items():
        mean_metric.labels(feature=feature).set(stats["mean"])
        stddev_metric.labels(feature=feature).set(stats["stddev"])
        outlier_metric.labels(feature=feature).set(stats["outliers"])

    # ✅ Push Class Distribution
    for species, percentage in class_counts.items():
        class_distribution.labels(species=iris_data["target_names"][species]).set(percentage)

    current_time = time.time()
    if current_time - last_push >= push_interval:
        try:
            push_to_gateway("localhost:9091", job="model_metrics", registry=registry)
            log_event("info", f"Metrics pushed at iteration {i}. RMSE={rmse:.4f}, MAE={mae:.4f} , Z_Score = {z_scores} , Mean_Value = {mean_value}")
            last_push = current_time
        except Exception as e:
            log_event("error", f"Error pushing metrics: {str(e)}")

    time.sleep(0.001)


C:\Users\91966\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ fc1 (Dense)                     │ (None, 10)             │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 193 (772.00 B)

 Trainable params: 193 (772.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - loss: 0.2849 - mae: 0.4832 - mse: 0.2849 - val_loss: 0.2700 - val_mae: 0.4750 - val_mse: 0.2700
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.2842 - mae: 0.4847 - mse: 0.2842 - val_loss: 0.2564 - val_mae: 0.4640 - val_mse: 0.2564
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2620 - mae: 0.4672 - mse: 0.2620 - val_loss: 0.2428 - val_mae: 0.4511 - val_mse: 0.2428
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.2540 - mae: 0.4603 - mse: 0.2540 - val_loss: 0.2319 - val_mae: 0.4395 - val_mse: 0.2319
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2497 - mae: 0.4558 - mse: 0.2497 - val_loss: 0.2245 - val_mae: 0.4313 - val_mse: 0.2245
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2422 - mae: 0.4484 - mse: 0.2422 - val_loss: 0.2187 - val_mae: 0.4262 - val_mse: 0.2187
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2422 - mae: 0.4495 - mse: 0.2422 - val_loss: 0.

In [78]:
# iris_data = load_iris()

# # ✅ Convert to Pandas DataFrame
# df = pd.DataFrame(iris_data['data'], columns=iris_data['feature_names'])

# # ✅ Print first few rows to verify
# print(df.head())
# print(df.columns[0])

df[feature]

0      0.2
1      0.2
2      0.2
3      0.2
4      0.2
      ... 
145    2.3
146    1.9
147    2.0
148    2.3
149    1.8
Name: petal width (cm), Length: 150, dtype: float64

In [ ]:
print("🔍 Column Names in Dataset:", iris_data.keys())
print(iris_data['feature_names'])